In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
df = pd.read_csv("CAA.csv")
y =df["text"]
size = df.shape
# Remove leading numbers followed by a period and space using regular expressions
cleaned_data = y.str.replace(r'^\d+\.\s', '')

# Convert the Series to a list
cleaned_data = cleaned_data.tolist()
cleaned_data = [entry for entry in cleaned_data if isinstance(entry, str)]
# Convert the list to a set to remove duplicates
unique_data = set(cleaned_data)

# Convert the set back to a list if needed
cleaned_data = list(unique_data)

# Print the unique tweets
# print(unique_data_list[:20])
# specific_words = ["farmer", "#farmerprotest", "farmer protest"]
 # Specify the words you want to check for
specific_words = ["caa","#caa"]

# Filter out entries that do not contain any of the specific words (case-insensitive)
filtered_data = [entry for entry in cleaned_data if any(word.lower() in entry.lower() for word in specific_words)]

# Print the filtered data
# for entry in filtered_data:
#     print(entry)
# type(filtered_data)
import re

# Define a regular expression pattern to match links or words containing "www."
pattern = r'\b(?:https?://|www\.)\S+\b'

# Remove links or words containing "www." from the data
new_cleaned_data = [re.sub(pattern, '', entry) for entry in filtered_data]

# Print the cleaned data
# for entry in new_cleaned_data:
#     print(entry)
def preprocess_text(text_list):
    preprocessed_texts = []
    for text in text_list:
        # Tokenization
        tokens = word_tokenize(text)
        # Convert to lowercase
        tokens = [token.lower() for token in tokens]
        # Remove punctuation
        tokens = [token for token in tokens if token not in string.punctuation]
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        preprocessed_texts.append(tokens)
    return preprocessed_texts

preprocessed_text_list = [' '.join(tokens) for tokens in preprocess_text(cleaned_data)]
# print(preprocessed_text_list)
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove non-alphanumeric characters and symbols except whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    return text
statements = preprocessed_text_list
cleaned_statements = [clean_text(statement) for statement in statements]
# nlp approach using textblob
from textblob import TextBlob

def label_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 1  # Positive sentiment
    elif analysis.sentiment.polarity == 0:
        return 0  # Neutral sentiment
    else:
        return -1  # Negative sentiment

# Example list of strings
# list_of_statements =cleaned_statements

# Label each statement
textblob_labels = [label_sentiment(statement) for statement in cleaned_statements]
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Create an instance of the SentimentIntensityAnalyzer class

text = cleaned_statements
# List to store sentiment labels
actual_labels_list_vader = []

# Analyze sentiment for each text
for text in text:
    # Analyze sentiment of the text
    scores = analyzer.polarity_scores(text)

    # Convert sentiment score to label
    if scores['compound'] >= 0.05:
        actual_labels_list_vader.append(1)  # Positive sentiment
    elif scores['compound'] <= -0.05:
        actual_labels_list_vader.append(-1) # Negative sentiment
    else:
        actual_labels_list_vader.append(0)   # Neutral sentiment

def analyze_sentiment_vader(text):
  """
  Analyzes the sentiment of a statement using VADER (Valence Aware Dictionary and sEntiment Reasoner).

  Args:
      text: The text statement for sentiment analysis.

  Returns:
      A list containing:
          - Sentiment score (float): A compound score between -1 (negative) and 1 (positive).
          - Sentiment label (int): 1 for positive, -1 for negative, 0 for neutral.
  """

  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(text)

  sentiment_label = 0
  if scores['compound'] >= 0.05:
    sentiment_label = 1  # Positive sentiment
  elif scores['compound'] <= -0.05:
    sentiment_label = -1  # Negative sentiment

  return sentiment_label

vedar_labels = actual_labels_list_vader
textblob_labels_final = textblob_labels[2000:]
vedar_labels_final = vedar_labels[2000:]
# len(vedar_labels)
# len(textblob_labels)
# 2000 elemets label stored
data_dict = {}

for statement, label in zip(cleaned_statements[:1999], textblob_labels[:1999]):
    data_dict[statement] = label

# Printing the dictionary
X_train = []
y_train = []

for statement, label in zip(cleaned_statements[:1999], textblob_labels[:1999]):
    X_train.append(statement)
    y_train.append(label)
    
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train_vectorized, y_train)
def analyze_sentiment_svm(text):
  """
  Analyzes the sentiment of a statement using the globally defined SVM model and TF-IDF vectorizer.

  Args:
      text: The text statement for sentiment analysis.

  Returns:
      The predicted sentiment label (e.g., 1 for positive, -1 for negative).
  """

  # Vectorize the text
  text_vectorized = vectorizer.transform([text])

  # Predict sentiment using the SVM model
  sentiment_label = svm_model.predict(text_vectorized)[0]

  return sentiment_label

unlabeled_data = cleaned_statements[2000:]
X_unlabeled_vectorized = vectorizer.transform(unlabeled_data)
# 3. Make predictions
predicted_labels = svm_model.predict(X_unlabeled_vectorized)

# Store the predicted labels in a list
predicted_labels_list = predicted_labels.tolist()
svm_labels_final = predicted_labels_list

# finding mode
from statistics import mode
from collections import Counter

# Function to find the mode of three elements
def find_mode(a, b, c):
    count = Counter([a, b, c])
    most_common = count.most_common(1)
    return most_common[0][0]

# List to store the mode of corresponding elements
final_labels = []

# Iterate through the lists and find the mode of each set of corresponding elements
for svm, vedar, textblob in zip(svm_labels_final, vedar_labels_final, textblob_labels_final):
    final_labels.append(find_mode(svm, vedar, textblob))

# Print the final labels
# print(final_labels)

# previous method

import matplotlib.pyplot as plt

# Count the occurrences of each label
label_counts = {-1: 0, 0: 0, 1: 0}
for label in final_labels:
    label_counts[label] += 1

# Plot the distribution
plt.bar(label_counts.keys(), label_counts.values(), color=['red', 'blue', 'green'])
plt.xlabel('Sentiment Label')
plt.ylabel('Frequency')
plt.title('Distribution of Sentiment Labels')
plt.xticks([-1, 0, 1], ['-1 (Negative)', '0 (Neutral)', '1 (Positive)'])
plt.show()

# Calculate percentages
total_statements = len(cleaned_statements)
percentage_label_counts = {label: (count / total_statements) * 100 for label, count in label_counts.items()}

# Plot the distribution
plt.bar(percentage_label_counts.keys(), percentage_label_counts.values(), color=['red', 'blue', 'green'])
plt.xlabel('Sentiment Label')
plt.ylabel('Percentage')
plt.title('Distribution of Sentiment Labels')
plt.xticks([-1, 0, 1], ['-1 (Negative)', '0 (Neutral)', '1 (Positive)'])
plt.show()

# Pie chart
plt.pie(percentage_label_counts.values(), labels=percentage_label_counts.keys(), autopct='%1.1f%%', colors=['red', 'blue', 'green'])
plt.title('Distribution of Sentiment Labels')
plt.show()

def analyze_sentiment_ensemble(text):
  """
  Analyzes sentiment of a statement using an ensemble of three models and returns the mode.

  Args:
      text: The text statement for sentiment analysis.

  Returns:
      The mode of sentiment labels (-1 for negative, 0 for neutral, 1 for positive)
          based on the results from the three models.
  """

  # Call sentiment analysis functions
  vader_result = analyze_sentiment_vader(text)  # Assuming it returns a sentiment label
  label_sentiment_result = label_sentiment(text)  # Assuming it returns a sentiment label
  svm_result = analyze_sentiment_svm(text)  # Assuming it returns a sentiment label

  # Store results
  results = [vader_result, label_sentiment_result, svm_result]

  # Count occurrences of each sentiment label
  from collections import Counter
  sentiment_counts = Counter(results)

  # Find the mode (most frequent sentiment)
  mode_sentiment = sentiment_counts.most_common(1)[0][0]

  return mode_sentiment



import speech_recognition as sr
import pyttsx3

# Initialize speech recognition and text-to-speech engines
recognizer = sr.Recognizer()
engine = pyttsx3.init()

def speak(text):
    """Speaks the given text using the text-to-speech engine."""
    engine.say(text)
    engine.runAndWait()

def listen():
    """Listens for user input using speech recognition."""
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for ambient noise
        print("Listening...")
        audio = recognizer.listen(source, timeout=10, phrase_time_limit=10)  # Listen for longer period

    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text.lower()  # Convert to lowercase for case-insensitive analysis
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None


def main():
    """Main loop for voice assistant interaction."""
    speak("Hello! I am Senti, your voice assistant. How can I help you today?")
    while True:
        text = listen()
        if text:
            if "stop" in text or "exit" in text:
                speak("Goodbye!")
                break
            sentiment = analyze_sentiment_ensemble(text)
            if sentiment == -1:
                speak("That sounds negative. Is there anything I can help you with?")
            elif sentiment == 0:
                speak("Seems like a neutral statement. Do you have anything else you'd like to say?")
            else:
                speak(f"That sounds positive! What can I do for you today?")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Asus\AppData\Local\Temp\ipykernel_8400\1985160743.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_data = y.str.replace(r'^\d+\.\s', '')


In [ ]:

print("Please enter some text:")
text = input()
print(analyze_sentiment_ensemble(text))